In [3]:
import sys
import os
import http.client
import json
import pandas as pd

In [4]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [5]:
def demanda_evolucion(input_anio, geo_ids):

    result = dict()
    try:
        anio = input_anio

        conexion = http.client.HTTPSConnection("apidatos.ree.es")

        headers = {
            "Accept": "application/json;",
            "Content-Type": "application/json",
            "Host": "apidatos.ree.es"
        }

        url = f"/es/datos/demanda/evolucion?start_date={input_anio}-01-01T00:00&end_date={input_anio}-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids={geo_ids}"

        conexion.request("GET", url, headers=headers)

        res = conexion.getresponse()
        data = res.read()

        dicc_datos = json.loads(data.decode("utf-8"))

        result = dicc_datos['included'][0]['attributes']['values']

    except:
        print(f"Error al hacer peticion api, en get_datos: {url}")
    return result

def datos_csv():
    list_anios=[2019,2021,2022]
    for x, y in cod_comunidades.items():
        for i in list_anios:
            diccionario =  demanda_evolucion(i, y)
            df_final = pd.DataFrame(diccionario)
            df_final.to_csv(f"../datos/{i}_demanda_evolucion_{x}_{y}.csv")

datos_csv()

In [6]:
def datos_demanda_generacion(input_anio, geo_ids):

    result = dict()
    dicc_datos = dict()
    try:
        anio = input_anio

        conexion = http.client.HTTPSConnection("apidatos.ree.es")

        headers = {
            "Accept": "application/json;",
            "Content-Type": "application/json",
            "Host": "apidatos.ree.es"
        }

        url = f"/es/datos/generacion/estructura-renovables?start_date={anio}-01-01T00:00&end_date={anio}-12-31T23:59&time_trunc=month&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={geo_ids}"
        conexion.request("GET", url, headers=headers)

        res = conexion.getresponse()

        if res.status == 200:
            data = res.read()
            dicc_datos = json.loads(data.decode("utf-8"))
            result = dicc_datos['included']

    except:
        print(f"Error al hacer peticion api, en get_datos_demanda_generacion: {url}")
    return result

def datos_csv():
     list_anios=[2019,2021,2022]
     for x, y in cod_comunidades.items():
        for i in list_anios:
            diccionario = datos_demanda_generacion(i, y)
            df_final = pd.DataFrame(diccionario)
            if df_final.shape[0] > 0:
                df_final.to_csv(f"../datos/{i}_generacion_estructura_{x}_{y}.csv")


datos_csv()